In [ ]:
import pandas as pd

names = pd.read_csv("names.csv")

In [ ]:
data = pd.read_json("/content/drive/MyDrive/Hammasir-Final/doctors.json")

In [ ]:
s_name = no_name["title"].str.split(" ", n=1)
f_name = s_name.str[0]
no_name["name"] = f_name

In [ ]:
data.loc[no_name.index, "name"] = no_name["name"]

In [ ]:
names["Gender"] = names["Gender"].replace("پسر", "M")
names["Gender"] = names["Gender"].replace("دختر", "F")
names.head()

In [ ]:
import re


def find_gender(data_name):
    # Escape special characters in cyka_name to treat it as a literal string
    escaped_name = re.escape(data_name)
    match = names[names["Name"].str.contains(escaped_name, regex=False)]
    if not match.empty:
        return match["Gender"].iloc[0]
    else:
        return None


data["gender"] = data["name"].apply(find_gender)
data.head()

In [ ]:
cent = data["centers"]
cities = []
for c in cent:
    city = c[0]["city_name"]
    cities.append(city)
    data["city"] = cities

In [ ]:
base_dataset = cyka[["city", "display_expertise", "gender"]]
base_dataset.head()

In [ ]:
base_dataset.to_csv("base_dataset.csv")